In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse
import gc
import lightfm

In [3]:
pd.set_option("display.max_columns", 1000)

In [4]:
from load_functions import *
from transform_functions import *
from score_functions import *
from i2i_functions import *
from attributes_recs import *
from learn_to_rank import *
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
test_users = pd.read_csv('test_users.csv', dtype=str, names=['user_id'],header=0)
users_df = load_users('users.csv', test_users)
orgs_df = load_orgs('organisations.csv')
combined_features = load_combined(["rubrics.csv","features.csv"], orgs_df=orgs_df)
test_users_with_city = users_df[users_df.in_test==1][['user_id','city']]
print(len(test_users),len(users_df))

16967 1252801


In [6]:
reviews = load_reviews("reviews.csv", users_df=users_df, orgs_df=orgs_df)
orgs_df=enrich_orgs(orgs_df, reviews)
print(f"{len(reviews)}")
reviews.head()

3640835


,user_id,org_id,rating,ts,aspects,good,user_city,in_test,org_city,travel
0,16998268288908323644,7184895086928047809,2.0,105,[],0,msk,0,msk,0
1,3121447338909258868,7184895086928047809,5.0,464,[],1,msk,0,msk,0
2,1970649778250883025,7184895086928047809,3.0,789,[],0,msk,0,msk,0
3,7554889464530643866,7184895086928047809,4.0,936,[],1,msk,0,msk,0
4,15907910894057053620,7184895086928047809,1.0,1143,[],0,msk,0,msk,0


In [7]:
test_users_df = test_users_with_city\
    .merge(reviews[reviews.good>0][['user_id','org_id']], on='user_id', how='left', indicator=True)
good_test_users = test_users_df[test_users_df["_merge"]=='both']\
    .groupby(['user_id','city'])\
    .agg(org_id=pd.NamedAgg("org_id", aggfunc=list)).reset_index()

empty_test_users = test_users_df[test_users_df["_merge"]=='left_only'][['user_id','city']]
empty_test_users['org_id'] = [[] for _ in empty_test_users.itertuples()]
final_test_users_df = pd.concat([good_test_users, empty_test_users], ignore_index=True)
del test_users_df
del good_test_users
del empty_test_users
print(len(final_test_users_df), len(test_users_with_city))

16967 16967


In [8]:
draft_train_reviews, draft_test_reviews = train_test_split(reviews, min_ts=1000)
print(len(draft_train_reviews), len(draft_test_reviews))

9556 9556


In [ ]:
train_pairs = draft_train_reviews.explode("org_id").drop(columns='city')
test_pairs = draft_test_reviews.explode("target").rename(columns={'target':'org_id'})

result, encs = prepare_reviews_i2i(reviews, orgs_df,
                                   train_pairs, test_pairs,
                                   min_reviews_per_user=2,
                                   min_org_reviews=5,
                                   min_travels_reviews=0,
                                   min_org_score=4.0)
may_be_in_draft = set(encs.orgs_enc.classes_)
should_be_in_draft = set(draft_test_reviews.target.explode())
print(len(result))
print(f"{len(may_be_in_draft)},{len(should_be_in_draft)},{len(may_be_in_draft&should_be_in_draft)/len(should_be_in_draft)}")

In [ ]:
pandarallel.initialize(progress_bar=False, nb_workers=1)
l2=1.0e5
m = reviews_matrix(result, encs)
cc_mat = m.T * m
del m
del cc_mat
ease_sim = ease_solution(cc_mat, l2=l2)
draft_ease_preds = i2i_predict(ease_sim.T, draft_train_reviews, encs, N=60)
print(f"MNAP: {mnap(draft_test_reviews, draft_ease_preds, N=20)}")
for N in [20,60]:
    print(f"{N} {recall(draft_test_reviews, draft_ease_preds, N=N).mean()}")
for t_size in [1,3,5,10,20,50]:
    draft_test_preds = draft_test_reviews[draft_test_reviews.target.str.len()<=t_size]
    print(f"<={t_size} {recall(draft_test_preds, draft_ease_preds).mean()}")

In [ ]:
del cc_mat
gc.collect()

In [ ]:
combined_enc = AttrEncoders(orgs_df, combined_features, 
                        colname='combined_id',
                        org_colname='combined_id')
combined_enc.build_attr_org_mat(ease_sim, encs)
pandarallel.initialize(progress_bar=False)
draft_combined_recs = combined_enc.attr_predict(draft_train_reviews, encs, N=60)
print(f"{mnap(draft_test_reviews, draft_combined_recs, N=N)}")
for N in [20,60]:
    print(f"{N} {recall(draft_test_reviews, draft_combined_recs, N=N).mean()}")
for t_size in [1,3,5,10,20]:
    draft_test_preds = draft_test_reviews[draft_test_reviews.target.str.len()<=t_size]
    print(f"<={t_size} {recall(draft_test_preds, draft_combined_recs).mean()}")

In [ ]:
draft_train_reviews.head()

In [ ]:
merged_preds = combine_preds(draft_train_reviews, draft_ease_preds, draft_combined_recs, min_len=2)
print(f"{N} {recall(draft_test_reviews, merged_preds, N=20).mean()}")
# pool_builder = PoolBuilder(orgs_df, reviews)
# final_train_data = pool_builder.build_big_pool(merged_preds, draft_test_reviews)

In [ ]:
gc.collect()

In [ ]:
train_ranking, valid_ranking = split_4_ranking(final_train_data)

In [ ]:
train_pool = train_data_to_pool(train_ranking)
valid_pool = train_data_to_pool(valid_ranking)

In [ ]:
def cb_params():
    return {
    'custom_metric': ['PFound:top=2'],
    'loss_function':"YetiRank",
    'metric_period': 50,
    'learning_rate': 0.2,
    'l2_leaf_reg': 10,
    'iterations': 10000
}

In [ ]:
model = CatBoost(cb_params())
model.fit(train_pool, plot=True, eval_set=valid_pool, verbose=False)

In [ ]:
pandarallel.initialize(progress_bar=False)
valid_preds = cb_predict(model, valid_pool, valid_ranking.groups, valid_ranking.org_ids)
print(f"mnap {mnap(draft_test_reviews, valid_preds, N=20)}")
print(f"recall {recall(draft_test_reviews, valid_preds, N=20).mean()}")

print(f"mnap {mnap(draft_test_reviews, draft_ease_preds, N=20)}")
print(f"recall {recall(draft_test_reviews, draft_ease_preds, N=20).mean()}")

## Final predicts

In [12]:
result, encs = prepare_reviews_i2i(reviews, orgs_df,
                                   None, None,
                                   min_reviews_per_user=2,
                                   min_org_reviews=5,
                                   min_travels_reviews=0,
                                   min_org_score=3.5)
pandarallel.initialize(progress_bar=False, nb_workers=1)
l2=1.0e5
m = reviews_matrix(result, encs)
cc_mat = m.T * m
final_ease_sim = ease_solution(cc_mat, l2=l2).T
del m
del cc_mat
final_draft_ease_preds = i2i_predict(final_ease_sim, final_test_users_df, encs, N=60)

combined_enc = AttrEncoders(orgs_df, combined_features, 
                        colname='combined_id',
                        org_colname='combined_id')
combined_enc.build_attr_org_mat(final_ease_sim, encs)
pandarallel.initialize(progress_bar=False)
final_draft_combined_recs = combined_enc.attr_predict(final_test_users_df, encs, N=60)
merged = combine_preds(final_test_users_df, final_draft_ease_preds, final_draft_combined_recs, min_len=1)
merged['target'] = merged.target.apply(lambda t: t[:20])
merged = fallback_with_top_recs(merged, orgs_df)
validate_preds(merged, orgs_df, users_df, N=20)
save_predictions(merged, "final_ans.csv")

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
All good


In [ ]:
merged_pairs = merged[['user_id','target']].explode('target')

In [ ]:
# final_pool = pool_builder.build_test_pool(merged)
# final_cb_pool = test_data_to_pool(final_pool)
# final_preds = cb_predict(model, final_cb_pool, final_pool.groups, final_pool.org_ids)
# validate_preds(final_preds, orgs_df, users_df, N=20)
save_predictions(final_preds, "reranked.csv")

In [ ]:
compare_preds(final_preds, draft_final_ease_preds)